# Somvanshi et al. (2020) Model Code <a name="top" />

## Table of Contents
1. [Instructions](#instructions)
    1. [Parameter Optimization Against TSST Data Sets](#TSSTInstructions)
    2. [Parameter Optimization Against Basal Data Sets](#basalInstructions)
    3. [Running Without Parameter Optimization](#noOptInstructions)
2. [Imports](#imports)
3. [Parameters and Initial Conditions](#params)
4. [Put Raw Data into Arrays](#rawdata)
   1. [Plot Data Sets](#plotdata)
5. [Model Function--Includes ODE Solver](#modelfunction)
6. [Cost Function Definition](#cost)
7. [Run the Optimization](#run)
8. [Save Output to File](#saveoutput)
9. [Compute Means and Std Devations of Parameters and Output as Table](#paramtable)
10. [Plots](#plots)
11. [No Optimization Run and Plot](#no-opt)
    1. [Plot the No-Optimization Simulation](#plotNoOpt)
12. [Dependencies](#dependencies)

## Instructions <a name="instructions"></a>

### Parameter Optimization Against TSST Data Sets <a name="TSSTInstructions" />

**Note:** To quickly run a cell (or a selection of cells), use the shortcut Shift+Enter (or you can also use the button labeled "Run" in the toolbar at the top).

To run simulations with parameter optimization against TSST data, there is no need to change any cells until the cost function cell. Simply run all cells up to the cell below the heading **Cost Function Definition**.

In order to run against a patient from the TSST data sets, simply change the second and fourth arguments to reflect the patient number and subject group. The subject groups are:

- nelson.melancholicACTH & nelson.melancholicCortisol (15 patients)
- nelson.atypicalACTH & nelson.atypicalCortisol (14 patients)
- nelson.neitherACTH & nelson.neitherCortisol (14 patients)
- nelson.healthyACTH & nelson.healthyCortisol (15 patients)

You could also run against the mean of all patients cortisol and ACTH concentration values by using `nelson.ACTH[:,1]` and `nelson.cortisol[:,1]`. Or you can run against the mean of any subgroup using `nelson.<subgroup name>Cortisol_mean[:,1]` and `nelson.<subgroup name>ACTH_mean[:,1]` (for instance `nelson.melancholicCortisol_mean[:,1]` & `nelson.melancholicACTH_mean[:,1]`). 

Note that the first column in each data set is the time steps, so indexing with `[:,0]` is referring to the time. These are the values we need to pass as the first (ACTH time steps) and third (cortisol time steps) arguments to the cost function.

The following are several examples of arguments you could pass the cost function with explanations:

- `nelson.melancholicACTH[:,0], nelson.melancholicACTH[:,1], nelson.melancholicCortisol[:,0], nelson.melancholicCortisol[:,1], simData`
    - The 1st patient in the Melancholic subgroup
- `nelson.atypicalACTH[:,0], nelson.atypicalACTH[:,14], nelson.atypicalCortisol[:,0], nelson.atypicalCortisol[:,14], simData`
    - The 14th patient in the Atypical subgroup
- `nelson.healthyACTH[:,0], nelson.healthyACTH[:,2], nelson.healthyCortisol[:,0], nelson.healthyCortisol[:,2], simData`
    - The 2nd patient in the Healthy Control group
- `nelson.ACTH[:,0], nelson.ACTH[:,1], nelson.cortisol[:,0], nelson.cortisol[:,1], simData`
    - The mean data set for all patients (depressed and control)
- `nelson.healthyACTH_mean[:,0], nelson.healthyACTH_mean[:,1], nelson.healthyCortisol_mean[:,0], nelson.healthyCortisol_mean[:,1], simData`
    - The mean of all control patients
    
In order to find out the actual Patient ID of the patient you are matching, try creating a new cell and entering the following command (using the subtype name and index number you used in the cost function arguments):

    print(melancholic_ids[1])
    
Next, you need to set the initial conditions for each ODE. In the cell directly below the heading **Run the Optimization**, you'll need to change the following line:

    y0 = [13.5, nelson.ACTH[0,1], 0.794555311165795, nelson.cortisol[0,1], 0, 0, 11.1479571299593, 20.1160322148203, 342.786973832252, 24.8798181706611, 31.3302385380763, 0.000129677801623061, 5897.54196590606, 25, 0.446309929849742, 1.00160367257638, 22.8093965060946, 1.09564141359877]
    
Depending on which data set you used, change `nelsonACTH` and `nelsonCortisol` to match. Then, depending on which patient you used, change `[0,1]` in both places to `[0,#]` where # is replaced with the number from above. 

Alternatively, if you used the mean of all patients, you'd leave it unchanged.
    
And for each of the subgroup means, you'd use (or simply replace Healthy with whichever other subgroup you used):

    y0 = [13.5, nelson.healthyACTH_mean[0,1], nelson.healthyCortisol_mean[0,1], 0, 0, 11.1479571299593, 20.1160322148203, 342.786973832252, 24.8798181706611, 31.3302385380763, 0.000129677801623061, 5897.54196590606, 25, 0.446309929849742, 1.00160367257638, 22.8093965060946, 1.09564141359877]
    
At this point, you are ready to run the optimization, so simply run the cells up to the heading **Save Output to File**. This may take some time, so while it is running you can move on to the next steps (if you run a cell while another is processing, it will add it to a queue).

The next few cells under the heading **Save Output to File** can be changed so that the filenames match the subgroup and patient you are running. For instance, the filename below is for the Melancholic subgroup patient with ID 3:

    df_t.to_excel(excel_writer = 'somvanshiModel_output/somvanshiModel-nelson-melancholic-patientID_3-timeSteps-5-
        iterations.xlsx')
    
The final step after saving the outputs is to plot the simulations against the real-world data. The cell under the heading **Plots** contains the code for this purpose. The lines of concern to ensure that you're showing the data set that you matched are (marked with comments in the actual code cell):
```
ax2.plot(nelson.ACTH[:,0], nelson.ACTH[:,1], label = "Nelson ACTH Data - Patient Mean", color = "orange")
ax3.plot(nelson.cortisol[:,0], nelson.cortisol[:,1], label = "Nelson Cortisol Data - Patient Mean", 
    color = "orange")
```
Here, change the first two arguments of each line to match exactly the arguments you used for the cost function. The labels can be changed to the patient subgroup and Patient ID you matched, also. And the filename for the figure can be changed in the last line of the cell:

    plt.savefig("somvanshiModel_output/somvanshiModel-nelson-melancholic-patientID_3-5-iterations-all-params-
        normalizedCost.png", dpi = 300)
    
### Parameter Optimization Against Basal Data Sets <a name="basalInstructions" />

Since these data sets have data points over a 24-hour period, rather than 2.35 hours, you will need to change the time interval over which the ODE solver integrates. To do this, go to the cell directly above the heading **Put Raw Data Into Arrays** and change the value of t_end to the following:

    t_end = 24.26

The reason you add the extra 0.26 hours is that you need to make sure that when you interpolate between your simulated data points the line covers every real-world data point so that you don't cause issues when computing the cost function (and the last data point for the Golier cortisol concentration data sets is at 24.25 hours).

After making this change, you need to again change the cost function arguments so that you are matching the basal data set in which you are interested. This time, however, you will also need to change the first and third arguments, because we need to tell the function the correct time steps for the data set.

First, choose which data set you wish to match. Here are the options:

- yehuda.controlCortisol
- yehuda.PTSDCortisol
- yehuda.depressedCortisol
- carroll.controlCortisol & carroll.controlACTH
- carroll.LCDepressedCortisol & carroll.LCDepressedACTH (LC = Low Cortisol)
- carroll.HCDepressedCortisol & carroll.HCDepressedACTH (HC = High Cortisol)
- golier.PTSDCortisol & golierPTSDACTH
- golier.nonPTSDTraumaExposedCortisol & golier.nonPTSDTraumaExposedACTH
- golier.nonPTSDNonExposedCortisol & golier.nonPTSDNonExposedACTH
- bremner.abusedPTSDCortisol
- bremner.nonAbusedPTSDCortisol
- bremner.nonAbusedNonPTSDCortisol

**Note:** To see what any of these data sets looks like, click on the **Plot Basal Data Sets** heading in the Table of Contents.

**Note Also:** These data sets all come in smoothed versions (each data point is set to the average of the nearest 5 points of the unsmoothed data). Also, the data sets by Carroll, Golier and Bremner also come in rearranged (or smoothed & rearranged) versions to match the starting time of the Yehuda data (10AM). To use any of these versions, simply append one of the following tags to the end of the data set name (before the indices): `_smooth`, `_rearr`, or `_rearr_smooth`.

First, I will cover what to do with data sets that contain both ACTH and cortisol values, and then afterwards I will cover using the Yehuda and Bremner data sets (which have only cortisol concentration data). For all of these data sets, the first column is the time step values. This means that if you take any of these arrays and index it with `[:,0]`, you are referring to the time steps. These are the values we need to pass as the first (ACTH time steps) and third (cortisol time steps) arguments to the cost function.

Then for the second and fourth arguments, you index the same data sets with `[:,1]` to mean the second column (which contains the mean concentration values for each patient group).

Here are a couple of examples showing arguments you can pass to the cost function:

- `carroll.controlACTH_smooth[:,0], carroll.controlACTH_smooth[:,1], carroll.controlCortisol_smooth[:,0], carroll.controlCortisol_smooth[:,1], simData`
    - The smoothed Control group mean for the Carroll data set
- `golier.nonPTSDTraumaExposedACTH[:,0], golier.nonPTSDTraumaExposedACTH[:,1], golier.nonPTSDTraumaExposedCortisol[:,0], golier.nonPTSDTraumaExposedCortisol[:,1], simData`
    - The Trauma-Exposed Control group mean for the Golier data set
    
In order to run simulations against data sets that do not include ACTH concentration data, you will need to change the name of the function to `costFun.SSE_cost_noACTH` and then simply do not include the two arguments for ACTH data. To use the Yehuda Control group data, this would look like:

    return costFun.SSE_cost_noACTH(yehuda.controlCortisol[:,0], yehuda.controlCortisol[:,1], simData)

You'll also need to change the initial conditions in the cell below the heading **Run the Optimization**, again. Use the name of the data set you used for the cost function, and change the indices to `[0,1]` for both. Here's an example (using the smoothed Carroll Control group data set):

    y0 = [13.5, carroll.controlACTH_smooth[0,1], carroll.controlCortisol_smooth[0,1], 0, 0, 11.1479571299593, 20.1160322148203, 342.786973832252, 24.8798181706611, 31.3302385380763, 0.000129677801623061, 5897.54196590606, 25, 0.446309929849742, 1.00160367257638, 22.8093965060946, 1.09564141359877]
    
Next, because you changed the length of the interval over which the ODE solver integrates, you'll need to change the arrays initialized in the next cell to be longer. For each of the last 4 lines in the cell, you'll need to replace the number 237 with 2428 (since we start at -0.01 and go until 24.26 with a step size of 0.01, that's 2428 steps total).

At this point, you're ready to run the parameter optimization, so run the next cell. Again, it may take a while, so you can start editing the remaining cells while you wait.

The cells under the heading **Save Output to File** should each have the filename changed to something that reflects the data set you're matching now. For instance, the Excel filename for sims_acth when matching the smoothed Carroll Control group would become:

    df_crh.to_excel(excel_writer = 'somvanshiModel_output/somvanshiModel-carroll-control-smooth-sims-crh-5-
        iterations.xlsx')
            
Finally, the cell under the heading **Plots** needs to have the same lines changed as described above. For instance, when matching the smoothed Carroll Control group, they would become:
```
ax2.plot(carroll.controlACTH_smooth[:,0], carroll.controlACTH_smooth[:,1], label = "Carroll Control ACTH Data", 
    color = "orange")
ax3.plot(carroll.controlCortisol_smooth[:,0], carroll.controlCortisol_smooth[:,1], label = "Carroll Control 
    Cortisol Data", color = "orange")
```
The data set names and indices here should match exactly the names and indices used as arguments to the cost function. Also, the last line of the cell can have the filename changed as described above to reflect which data set you are plotting.

### Running Without Parameter Optimization <a name="noOptInstructions" />

To run the model with any set of paramaters you desire, without optimization, you can use the cells under the heading **No Optimization Run**. Set the parameters, initial conditions and time interval you want to use by changing the values defined in the section **Parameters and Initial Conditions**. Then you can run the cell immediately after the heading **Plot the No-Optimization Simulation** to plot the simulation values. Currently, the values for time interval are set to run the simulation for 124 hours and then plot only the final 24 hours.

[Back to Top](#top)

## Imports <a name="imports"></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy.optimize as sco
from scipy import optimize
from scipy.interpolate import interp1d
import mpld3
from tabulate import tabulate
import pandas as pd
from HPAmodeling import DEsolver, costFun
from HPAmodeling.dataImport import data

[Back to Top](#top)

## Parameters and Initial Conditions <a name="params"></a>

In [ ]:
# initial conditions
# order: CRH, ACTH, StARp, CORT, Dex1, Dex2, CORTp, GR_mrna, GR_prot, GR_cyt, GR, LPS, Phg, Phg1, TGF, TNF, IL10, IL6

# based on: authors' code
y0=[17.4896364862443,4.82352867112865,0.794555311165795,10.8943173854454,0,0,11.1479571299593,20.1160322148203,342.786973832252,24.8798181706611,31.3302385380763,0.000129677801623061,5897.54196590606,25,0.446309929849742,1.00160367257638,22.8093965060946,1.09564141359877]

In [ ]:
# authors' listed parameter values
p = 0.0    # LPS injection
r = 0      # change this to vary GR sensitivity
p1 = 0     # change this to 1 to introduce LPS for IC50 test
s0 = 1     # change this to 0 to disconnect HPA with Inflammation for IC50 test
#u=1       # make this 0.1 for simulating IC50, else 1

Ki = 1     # inhibitory constant
n = 1*(1+r)# sensitivity of GRs
dex = 1    # dexamethasone
R = 1      # cytokine effect on HPA axis
Q = 1      # GR negative feedback strength
Ki1 = Ki   # the authors also have 1*(1+kt) commented out here, as an option
nj = 1*(1+r) # Hill coefficient for TGF, changes response of inflammatory cytokines to GRs
st = 0     # the authors define this one as 0*((t>0) and (t<5000)), which
#          # seems unnecessary, as that will always just return 0

# parameters listed in the model function of the authors' code
lp = p
It = lp*10e7 # this is defined only for t between 712 and 712.1 in the authors' code
#            # but it will always be 0 here anyway, since we have set p=0
#            # these parameters have to do with the LPS dosing, which we
#            # are not doing right now

# these next 4 lines are for changing dexamethasone dosage
dx = 0
dex = dx*10 # comment this to run IC50 test
d = 0
Dex = dex    # the authors only set this when t is between 696+d and 696.25+d
#            # but since we just have dex = 0 anyway, it doesn't change anything

K_strs = 10
Ki_GR1 = 1.2*Ki
n1 = n
n2 = 2
k_crh = 0.096
V_acth1 = 1.4634
k_acth = 0.4312
k_cort = 0.99
V_cort = 5.67
k_dex1 = 1.95
V_crh = 0.0667*(1/R)
V_acth2 = 11.2
Km_tnf3 = 40*R
V_starp2 = 15
k_starp = 0.45
V_starp1 = 0.0225
Kil6 = 100
k_dex2 = 0.25
V_dex2 = 1.15
Ki_GR = 50
V_GRmrna = 3.4
km_GR1 = 25*Ki
kon = 0.00329
k_GRprot = 0.0572
krt = 0.63
kre = 0.57
k_GRmrna = 0.1124
V_GRprot = 1.2
fr = 0.49
tc = 0.1401
ki_tnf = 1000
n3 = 2*n
n4 = 2
fac = 16.65

## Inflammatory Pathway Parameters from Authors' Code
# The authors call it "Validation paramset 2 for 0.4 ng/kg @ 2PM"

k_lps = 2.7e-5
V_phg1 = 4.9956e7
V_phg2 = 12.949
Km_tnf1 = 1693.9509
Ki_tgf1 = 0.00721
Ki_IL10 = 7.384
k_phg = 1.439
V_tgf1 = 0.15625e-8
k_tgf = 0.0635
V_tnf1 = 25.5194
Km_phg1 = 412500
Ki_tgf2 = 0.143
V_tnf2 = 106542
Km_tnf2 = 123.96
k_tnf = 1.25
Km_il61 = 80
Km_phg2 = 161012
k_il10 = 1.6
V_il102 = 2.1938e3
Km_tgf3 = 0.76
V_il101 = 1.3374e3
Ki_il102 = 23.636
V_il62 = 5e5
Km_il6tnf = 339.164
Km_phg3 = 11e6
V_il61 = 5.5e4
k_il6 = 1.625
tp = 1.5
tnf_b = 1.25
il6_b = 1.5
V_tgf2 = 0.5
Km_GR2 = 500*Ki1
n5 = 1*nj
n6 = 4
n7 = 2
n8 = 2
n9 = 6
n10 = 2

omega = 2*np.pi/24 # this parameter is used in the circadian functions

# save an array for use when not optimizing parameters
# we will start with only optimizing Ki and r, the parameters that have 
# to do with strength of GR feedback
# parameters skipped due to being proportional to another that is already
# optimized: Km_GR2, n5, Ki_GR1, n1
# need to take out: Km_GR1
authors_params = [Ki, r, K_strs, n2, k_crh, V_acth1, k_acth, k_cort, V_cort, k_dex1, V_crh, V_acth2, Km_tnf3, V_starp2, k_starp, V_starp1, Kil6, k_dex2, V_dex2, Ki_GR, V_GRmrna, kon, k_GRprot, krt, kre, k_GRmrna, V_GRprot, fr, tc, ki_tnf, n3, n4, fac, k_lps, V_phg1, V_phg2, Km_tnf1, Ki_tgf1, Ki_IL10, k_phg, V_tgf1, k_tgf, V_tnf1, Km_phg1, Ki_tgf2, V_tnf2, Km_tnf2, k_tnf, Km_il61, Km_phg2, k_il10, V_il102, Km_tgf3, V_il101, Ki_il102, V_il62, Km_il6tnf, Km_phg3, V_il61, k_il6, tp, tnf_b, il6_b, V_tgf2, n6, n7, n8, n9, n10]

In [ ]:
# compute bounds based on +- 10%
bound = kon
print(bound - bound*.1)
print(bound + bound*.1)

In [ ]:
# bounds for parameter optimization
# starting with +- 10% since we do not have published ranges in the paper
# order is: Ki, r, K_strs, n2, k_crh, V_acth1, k_acth, k_cort, V_cort, k_dex1, V_crh, V_acth2, Km_tnf3, V_starp2, k_starp, V_starp1, Kil6, k_dex2, V_dex2, Ki_GR, V_GRmrna, km_GR1, kon, k_GRprot, krt, kre, k_GRmrna, V_GRprot, fr, tc, ki_tnf, n3, n4, fac
bounds = [(0.1, 4), (-0.9, 3), (9., 11.), (1.8, 2.2), (0.0864, 0.1056), (1.3170600000000001, 1.60974), (0.38808000000000004, 0.47432), (0.891, 1.089), (5.103, 6.237), (1.755, 2.145), (0.06002999999999999, 0.07336999999999999), (10.08, 12.319999999999999), (36., 44.), (13.5, 16.5), (0.405, 0.495), (0.02025, 0.024749999), (90., 110.), (0.225, 0.275), (1.035, 1.265), (45., 55.), (3.06, 3.73999), (0.002961, 0.003619), (0.05148, 0.06292), (0.567, 0.693), (0.513, 0.627), (0.10116, 0.12364), (1.08, 1.32), (0.441, 0.539), (0.12609, 0.15411), (900., 1100.), (1.8, 2.2), (1.8, 2.2), (14.985, 18.315), (2.43e-5, 2.97e-5), (44960400., 54951600.), (11.6541, 14.2439), (1524.55581, 1863.34599), (0.006489, 0.007931), (6.6456, 8.1224), (1.2951, 1.5829), (1.40625e-9, 1.71845e-9), (0.05715, 0.06985), (22.96746, 28.07134), (371250., 453750.), (0.1287, 0.1573), (95887.8, 117196.2), (111.564, 136.356), (1.125, 1.375), (72., 88.), (144910.8, 177113.2), (1.44, 1.76), (1974.42, 2413.18), (0.684, 0.836), (1203.66, 1471.14), (21.2724, 25.9996), (450000., 550000.), (305.2476, 373.0804), (9900000., 12100000.), (49500., 60500.), (1.4625, 1.7875), (1.35, 1.65), (1.125, 1.375), (1.35, 1.65), (0.45, 0.55), (3.6, 4.4), (1.8, 2.2), (1.8, 2.2), (5.4, 6.6), (1.8, 2.2)]

In [ ]:
# define time interval for integration

# time interval and step definition
# all data sets end on 1440.0 or earlier except the Golier cortisol sets,
# they end on 1455.0, so I should set t_end = 1455.01 when matching them
#t_start = -0.01
#t_end = 24.26
#t_step = 0.01

# for matching Nelson data, use these values of t_start, t_end and t_step
t_start = -0.01
t_end = 2.35
t_step = 0.01

[Back to Top](#top)

## Put Raw Data into Arrays <a name="rawdata"></a>

In [ ]:
# Create an instance of the data class for each data set contained in the HPAmodeling library, and set the time
# scale to hours.
yehuda = data("yehuda", "hours")
carroll = data("carroll", "hours")
golier = data("golier", "hours")
bremner = data("bremner", "hours")
nelson = data("nelson", "hours")

### Plot Data Sets <a name="plotdata"></a>

In [ ]:
%matplotlib inline
font = {'size'   : 18}
matplotlib.rc('font', **font)

fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, figsize = (20,20))

ax1.plot(yehuda.controlCortisol[:,0], yehuda.controlCortisol[:,1], label = "Control Group Cortisol")
ax1.plot(yehuda.controlCortisol_smooth[:,0], yehuda.controlCortisol_smooth[:,1], label = "Control Group Cortisol - Smoothed")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(yehuda.PTSDCortisol[:,0], yehuda.PTSDCortisol[:,1], label = "PTSD Group Cortisol")
ax2.plot(yehuda.PTSDCortisol_smooth[:,0], yehuda.PTSDCortisol_smooth[:,1], label = "PTSD Group Cortisol - Smoothed")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(yehuda.depressedCortisol[:,0], yehuda.depressedCortisol[:,1], label = "Depression Group Cortisol")
ax3.plot(yehuda.depressedCortisol_smooth[:,0], yehuda.depressedCortisol_smooth[:,1], label = "Depression Group Cortisol - Smoothed")
ax3.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax3.legend(loc="lower right", shadow = True, fancybox = True)


In [ ]:
#mpld3.enable_notebook()
%matplotlib inline

font = {'size'   : 20}
matplotlib.rc('font', **font)
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows = 4, figsize = (25,25))

ax1.plot(carroll.controlCortisol_rearr[:,0], carroll.controlCortisol_rearr[:,1], 'b', label = "Control")
ax1.plot(carroll.HCDepressedCortisol_rearr[:,0], carroll.HCDepressedCortisol_rearr[:,1], 'r', label = "High Cortisol Depressed")
ax1.plot(carroll.controlCortisol_rearr_smooth[:,0], carroll.controlCortisol_rearr_smooth[:,1], label = "Control - Smoothed")
ax1.plot(carroll.HCDepressedCortisol_rearr_smooth[:,0], carroll.HCDepressedCortisol_rearr_smooth[:,1], label = "High Cortisol Depressed - Smoothed")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax1.legend(loc="upper right", shadow = True, fancybox = True)

ax2.plot(carroll.controlCortisol_rearr[:,0], carroll.controlCortisol_rearr[:,1], 'b', label = "Control")
ax2.plot(carroll.LCDepressedCortisol_rearr[:,0], carroll.LCDepressedCortisol_rearr[:,1], 'g', label = "Low Cortisol Depressed")
ax2.plot(carroll.controlCortisol_rearr_smooth[:,0], carroll.controlCortisol_rearr_smooth[:,1], label = "Control - Smoothed")
ax2.plot(carroll.LCDepressedCortisol_rearr_smooth[:,0], carroll.LCDepressedCortisol_rearr_smooth[:,1], label = "Low Cortisol Depressed - Smoothed")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (micrograms/dL)")
ax2.legend(loc="upper right", shadow = True, fancybox = True)

ax3.plot(carroll.controlACTH_rearr[:,0], carroll.controlACTH_rearr[:,1], 'b', label = "Control")
ax3.plot(carroll.HCDepressedACTH_rearr[:,0], carroll.HCDepressedACTH_rearr[:,1], 'r', label = "High Cortisol Depressed")
ax3.plot(carroll.controlACTH_rearr_smooth[:,0], carroll.controlACTH_rearr_smooth[:,1], label = "Control - Smoothed")
ax3.plot(carroll.HCDepressedACTH_rearr_smooth[:,0], carroll.HCDepressedACTH_rearr_smooth[:,1], label = "High Cortisol Depressed - Smoothed")
ax3.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax3.legend(loc="upper right", shadow = True, fancybox = True)

ax4.plot(carroll.controlACTH_rearr[:,0], carroll.controlACTH_rearr[:,1], 'b', label = "Control")
ax4.plot(carroll.LCDepressedACTH_rearr[:,0], carroll.LCDepressedACTH_rearr[:,1], 'g', label = "Low Cortisol Depressed")
ax4.plot(carroll.controlACTH_rearr_smooth[:,0], carroll.controlACTH_rearr_smooth[:,1], label = "Control - Smoothed")
ax4.plot(carroll.LCDepressedACTH_rearr_smooth[:,0], carroll.LCDepressedACTH_rearr_smooth[:,1], label = "Low Cortisol Depressed - Smoothed")
ax4.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax4.legend(loc="upper right", shadow = True, fancybox = True)

In [ ]:
%matplotlib inline

fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(nrows = 6, figsize = (15,20))

ax1.plot(golier.PTSDCortisol_rearr_smooth[:,0], golier.PTSDCortisol_rearr_smooth[:,1], label = "Trauma Exposed PTSD Cortisol - Smoothed")
ax1.plot(golier.PTSDCortisol_rearr[:,0], golier.PTSDCortisol_rearr[:,1], label = "Trauma Exposed PTSD Cortisol")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (mg/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(golier.nonPTSDTraumaExposedCortisol_rearr_smooth[:,0], golier.nonPTSDTraumaExposedCortisol_rearr_smooth[:,1], label = "Trauma Exposed Non-PTSD Cortisol - Smoothed")
ax2.plot(golier.nonPTSDTraumaExposedCortisol_rearr[:,0], golier.nonPTSDTraumaExposedCortisol_rearr[:,1], label = "Trauma Exposed Non-PTSD Cortisol")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (mg/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(golier.nonPTSDNonExposedCortisol_rearr_smooth[:,0], golier.nonPTSDNonExposedCortisol_rearr_smooth[:,1], label = "Non-Exposed Non-PTSD Cortisol - Smoothed")
ax3.plot(golier.nonPTSDNonExposedCortisol_rearr[:,0], golier.nonPTSDNonExposedCortisol_rearr[:,1], label = "Non-Exposed Non-PTSD Cortisol")
ax3.set(xlabel="Time (hours)", ylabel="Cortisol (mg/dL)")
ax3.legend(loc="lower right", shadow = True, fancybox = True)

ax4.plot(golier.PTSDACTH_rearr_smooth[:,0], golier.PTSDACTH_rearr_smooth[:,1], label = "Trauma Exposed PTSD ACTH - Smoothed")
ax4.plot(golier.PTSDACTH_rearr[:,0], golier.PTSDACTH_rearr[:,1], label = "Trauma Exposed PTSD ACTH")
ax4.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax4.legend(loc="lower right", shadow = True, fancybox = True)

ax5.plot(golier.nonPTSDTraumaExposedACTH_rearr_smooth[:,0], golier.nonPTSDTraumaExposedACTH_rearr_smooth[:,1], label = "Trauma Exposed Non-PTSD ACTH - Smoothed")
ax5.plot(golier.nonPTSDTraumaExposedACTH_rearr[:,0], golier.nonPTSDTraumaExposedACTH_rearr[:,1], label = "Trauma Exposed Non-PTSD ACTH")
ax5.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax5.legend(loc="lower right", shadow = True, fancybox = True)

ax6.plot(golier.nonPTSDNonExposedACTH_rearr_smooth[:,0], golier.nonPTSDNonExposedACTH_rearr_smooth[:,1], label = "Non-Exposed Non-PTSD ACTH - Smoothed")
ax6.plot(golier.nonPTSDNonExposedACTH_rearr[:,0], golier.nonPTSDNonExposedACTH_rearr[:,1], label = "Non-Exposed Non-PTSD ACTH")
ax6.set(xlabel="Time (hours)", ylabel="ACTH (pg/mL)")
ax6.legend(loc="lower right", shadow = True, fancybox = True)


In [ ]:
%matplotlib inline

fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, figsize = (15,15))

ax1.plot(bremner.abusedPTSDCortisol_rearr_smooth[:,0], bremner.abusedPTSDCortisol_rearr_smooth[:,1], label = "Abused PTSD Cortisol - Smoothed")
ax1.plot(bremner.abusedPTSDCortisol_rearr[:,0], bremner.abusedPTSDCortisol_rearr[:,1], label = "Abused PTSD Cortisol")
ax1.set(xlabel="Time (hours)", ylabel="Cortisol (microg/dL)")
ax1.legend(loc="lower right", shadow = True, fancybox = True)

ax2.plot(bremner.nonAbusedPTSDCortisol_rearr_smooth[:,0], bremner.nonAbusedPTSDCortisol_rearr_smooth[:,1], label = "Non-Abused PTSD Cortisol - Smoothed")
ax2.plot(bremner.nonAbusedPTSDCortisol_rearr[:,0], bremner.nonAbusedPTSDCortisol_rearr[:,1], label = "Non-Abused PTSD Cortisol")
ax2.set(xlabel="Time (hours)", ylabel="Cortisol (microg/dL)")
ax2.legend(loc="lower right", shadow = True, fancybox = True)

ax3.plot(bremner.nonAbusedNonPTSDCortisol_rearr_smooth[:,0], bremner.nonAbusedNonPTSDCortisol_rearr_smooth[:,1], label = "Non-Abused Non-PTSD Cortisol - Smoothed")
ax3.plot(bremner.nonAbusedNonPTSDCortisol_rearr[:,0], bremner.nonAbusedNonPTSDCortisol_rearr[:,1], label = "Non-Abused Non-PTSD Cortisol")
ax3.set(xlabel="Time (hours)", ylabel="Cortisol (microg/dL)")
ax3.legend(loc="lower left", shadow = True, fancybox = True)


In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (15, 15))

ax1.plot(nelson.ACTH[:,0], nelson.ACTH[:,1])
ax2.plot(nelson.cortisol[:,0], nelson.cortisol[:,1])

[Back to Top](#top)

## Model Function--Includes ODE Solver <a name="modelfunction"></a>

In [ ]:
def model(params, ics):
    def ode_system(t, y):
        
        dy = np.zeros(18)
        
        #[Ki, r, K_strs, n2, k_crh, V_acth1, k_acth, k_cort, V_cort, k_dex1, V_crh, V_acth2, Km_tnf3, V_starp2, k_starp, V_starp1, Kil6, k_dex2, V_dex2, Ki_GR, V_GRmrna, km_GR1, kon, k_GRprot, krt, kre, k_GRmrna, V_GRprot, fr, tc, ki_tnf, n3, n4, fac] = params
        [Ki, r, K_strs, n2, k_crh, V_acth1, k_acth, k_cort, V_cort, k_dex1, V_crh, V_acth2, Km_tnf3, V_starp2, k_starp, V_starp1, Kil6, k_dex2, V_dex2, Ki_GR, V_GRmrna, kon, k_GRprot, krt, kre, k_GRmrna, V_GRprot, fr, tc, ki_tnf, n3, n4, fac, k_lps, V_phg1, V_phg2, Km_tnf1, Ki_tgf1, Ki_IL10, k_phg, V_tgf1, k_tgf, V_tnf1, Km_phg1, Ki_tgf2, V_tnf2, Km_tnf2, k_tnf, Km_il61, Km_phg2, k_il10, V_il102, Km_tgf3, V_il101, Ki_il102, V_il62, Km_il6tnf, Km_phg3, V_il61, k_il6, tp, tnf_b, il6_b, V_tgf2, n6, n7, n8, n9, n10] = params
        
        cir_current = cir(t)
        cir2_current = cir2(t, cir_current, y[10])
        
        # CRH
        dy[0] = K_strs*(1 + st)*((Ki_GR1**n1)/(Ki_GR1**n1 + y[10]**n1))*cir_current*(1 + V_crh * y[15]) - k_crh*y[0]
        
        # ACTH
        dy[1] = V_acth1*y[0]*((Ki_GR1**n1)/(Ki_GR1**n1 + y[10]**n1))*(1 + V_acth2*((y[15]**n2)/(Km_tnf3**n2 + y[15]**n2))) - k_acth*y[1]
        
        # StAR Protein
        dy[2] = s0*(V_starp1*(y[1]*cir2_current)*(1 + V_starp2*((y[15]**n2)/(Km_tnf3**n2 + y[15]**n2))*(Kil6/(Kil6 + y[17]))) - k_starp*y[2])
        
        # Cortisol
        dy[3] = (V_cort*y[2] - k_cort*y[3])
        
        # Dexamethasone kinetics
        dy[4] = Dex - k_dex1*y[4]
        dy[5] = V_dex2*y[4] - k_dex2*y[5]
        
        # Peripheral Cortisol
        dy[6] = (1/tc)*(y[3] + fac*y[5] - y[6])
        
        # GR mRNA
        dy[7] = V_GRmrna*(1 - ((y[10]**n3)/(km_GR1**n3 + y[10]**n3))) - k_GRmrna*y[7]
        
        # GR protein
        dy[8] = V_GRprot*y[7] + fr*kre*y[10] - kon*y[6]*y[8] - k_GRprot*y[8]
        
        # GR-Cortisol complex in cytosol
        dy[9] = kon*y[6]*y[8] - krt*y[9]*((ki_tnf**n4)/(y[15]**n4 + ki_tnf**n4))
        
        # Nuclear GR
        dy[10] = krt*y[9]*((ki_tnf**n4)/(y[15]**n4 + ki_tnf**n4)) - kre*y[10]
        
        # LPS
        dy[11] = 1e-7*(1 + It) - k_lps*y[11]*y[12]
        
        # Phagocytes
        dy[12] = V_phg1*((1 + (V_phg2*y[15]/(Km_tnf1 + y[15])))*(Ki_tgf1/(Ki_tgf1 + y[14]))*(Ki_IL10/(Ki_IL10 + y[16])))*y[11] - k_phg*y[12]
        dy[13] = (1./tp)*(y[12] - y[13])
        
        # Transforming Growth Factor (TGF)
        dy[14] = V_tgf1*y[12] + (V_tgf2*(y[10]**n5)/(Km_GR2**n5 + y[10]**n5)) - k_tgf*y[14]
        
        # Tumor Necrosis Factor (TNF)
        dy[15] = tnf_b + (y[12]/(Km_phg1 + y[12]))*(V_tnf1 + V_tnf2*(y[15]/(Km_tnf2 + y[15])))*((Ki_tgf2**n6)/(Ki_tgf2**n6 + y[14]**n6))*(1 - ((y[17]**n7)/(Km_il61**n7 + y[17]**n7))) - k_tnf*y[15]
        
        # Interleukin 10 (IL-10)
        dy[16] = (V_il101*(y[13]**n8)/(y[13]**n8 + Km_phg2**n8)) + (V_il102*(y[14]**n9)/(Km_tgf3**n9 + y[14]**n9)) - k_il10*y[16]
        
        # Interleukin 6 (IL-6)
        dy[17] = il6_b + (y[13]/(Km_phg3 + y[13]))*(V_il61 + V_il62*((y[15] + y[17])**n10)/(Km_il6tnf**n10 + (y[15] + y[17])**n10))*(Ki_il102/(Ki_il102 + y[16])) - k_il6*y[17]
        
        return dy
    
    def cir(t):
        return 2*(1 + np.cos(omega*t))
    
    def cir2(t, cir, GR):
        return (4-cir)*(Ki_GR/(Ki_GR + GR))
    
    # Call the solve() function from my DEsolver module, and pass all of the information it needs.
    # Arguments are as follows: ODE function to solve, array of initial conditions, start time, step size, end time
    timeSeries = DEsolver.solve(ode_system, ics, t_start, t_step, t_end)
    return timeSeries

[Back to Top](#top)

## Cost Function Definition <a name="cost"></a>

In [ ]:
def cost_fun(params):
    simData = model(params, y0)
    
    return costFun.SSE_cost(nelson.ACTH[:,0], nelson.ACTH[:,1], nelson.cortisol[:,0], nelson.cortisol[:,1], simData)

[Back to Top](#top)

## Run the Optimization <a name="run"></a>

In [ ]:
# initial conditions
# order: CRH, ACTH, StARp, CORT, Dex1, Dex2, CORTp, GR_mrna, GR_prot, GR_cyt, GR, LPS, Phg, Phg1, TGF, TNF, IL10, IL6

# based on: golierPTSD
y0 = [13.5, nelson.ACTH[0,1], 0.794555311165795, nelson.cortisol[0,1], 0, 0, 11.1479571299593, 20.1160322148203, 342.786973832252, 24.8798181706611, 31.3302385380763, 0.000129677801623061, 5897.54196590606, 25, 0.446309929849742, 1.00160367257638, 22.8093965060946, 1.09564141359877]

In [ ]:
# number of times to run the optimization
n = 5

# define an array to hold the population of parameter vectors
opt_pars = np.zeros((n, len(bounds)+1))

# initialize arrays to save simulation cortisol and ACTH data from each optimization
sims_crh = np.zeros((237, n))
#sims_crh_100 = np.zeros((237, n))
sims_acth = np.zeros((237, n))
#sims_acth_100 = np.zeros((237, n))
sims_starp = np.zeros((237, n))
sims_cort = np.zeros((237, n))
#sims_cort_100 = np.zeros((237, n))
sims_dex1 = np.zeros((237, n))
sims_dex2 = np.zeros((237, n))
sims_cortp = np.zeros((237, n))
sims_gr_mrna = np.zeros((237, n))
sims_gr_prot = np.zeros((237, n))
sims_gr_cyt = np.zeros((237, n))
sims_gr = np.zeros((237, n))
#sims_gr_100 = np.zeros((237, n))
sims_lps = np.zeros((237, n))
sims_phg = np.zeros((237, n))
sims_phg1 = np.zeros((237, n))
sims_tgf = np.zeros((237, n))
sims_tnf = np.zeros((237, n))
sims_il10 = np.zeros((237, n))
sims_il6 = np.zeros((237, n))


In [ ]:
%%time

# loop n times, running the optimization each time
for i in range(0,n):
    
    print(f"Optimization Run #{i+1}")
    
    # call the differential evolution optimization function on the cost function
    res = sco.differential_evolution(cost_fun, bounds, maxiter = 999, disp = True, popsize = 1)
    
    # alternatively, we can run the SHGO algorithm with the sampling_method = "sobol" flag to do global
    #     optimization with reporting all local minima, as well
    #res = sco.shgo(cost_fun, bounds, callback=callback_fun(*shgo_iter_steps), options = {"f_min": 0.1, "maxiter": None, "minimize_every_iter": True, "local_iter": False, "disp": True}, iters = 3)
    #res = sco.basinhopping(cost_fun, x0, niter = 1000)
    #res = sco.dual_annealing(cost_fun, bounds)
    
    # plug the optimized parameters into the solver
    optimizedSimData = model(res.x, y0)
    # save CRH, cortisol and ACTH data into sims arrays
    sims_crh[:,i] = optimizedSimData[:,1]
    #sims_crh_100[:,i] = optimizedSimData[-237:,1]
    sims_acth[:,i] = optimizedSimData[:,2]
    #sims_acth_100[:,i] = optimizedSimData[-237:,2]
    sims_starp[:,i] = optimizedSimData[:,3]
    sims_cort[:,i] = optimizedSimData[:,4]
    #sims_cort_100[:,i] = optimizedSimData[-237:,4]
    sims_dex1[:,i] = optimizedSimData[:,5]
    sims_dex2[:,i] = optimizedSimData[:,6]
    sims_cortp[:,i] = optimizedSimData[:,7]
    sims_gr_mrna[:,i] = optimizedSimData[:,8]
    sims_gr_prot[:,i] = optimizedSimData[:,9]
    sims_gr_cyt[:,i] = optimizedSimData[:,10]
    sims_gr[:,i] = optimizedSimData[:,11]
    #sims_gr_100[:,i] = optimizedSimData[-237:,11]
    sims_lps[:,i] = optimizedSimData[:,12]
    sims_phg[:,i] = optimizedSimData[:,13]
    sims_phg1[:,i] = optimizedSimData[:,14]
    sims_tgf[:,i] = optimizedSimData[:,15]
    sims_tnf[:,i] = optimizedSimData[:,16]
    sims_il10[:,i] = optimizedSimData[:,17]
    sims_il6[:,i] = optimizedSimData[:,18]
    
    # save the cost function values and optimized parameters for each iteration into the array opt_pars
    opt_pars[i,0] = res.fun
    opt_pars[i,1:] = res.x

[Back to Top](#top)

## Save Output to File <a name="saveoutput"></a>

In [ ]:
df_t = pd.DataFrame(optimizedSimData[:,0])
df_gr = pd.DataFrame(sims_gr)
df_cort = pd.DataFrame(sims_cort)
df_acth = pd.DataFrame(sims_acth)
df_crh = pd.DataFrame(sims_crh)

df_t.to_excel(excel_writer = 'somvanshiModel_output/somvanshiModel-nelson-patientMean-timeSteps-5-iterations-all-params-normalizedCost.xlsx')
df_gr.to_excel(excel_writer = 'somvanshiModel_output/somvanshiModel-nelson-patientMean-sims-gr-5-iterations-all-params-normalizedCost.xlsx')
df_cort.to_excel(excel_writer = 'somvanshiModel_output/somvanshiModel-nelson-patientMean-sims-cort-5-iterations-all-params-normalizedCost.xlsx')
df_acth.to_excel(excel_writer = 'somvanshiModel_output/somvanshiModel-nelson-patientMean-sims-acth-5-iterations-all-params-normalizedCost.xlsx')
df_crh.to_excel(excel_writer = 'somvanshiModel_output/somvanshiModel-nelson-patientMean-sims-crh-5-iterations-all-params-normalizedCost.xlsx')


In [ ]:
np.savetxt('somvanshiModel_output/somvanshiModel-nelson-patient50-opt-pars-5-iterations-all-params.txt', opt_pars)

In [ ]:
np.savetxt('somvanshiModel_output/somvanshiModel-nelson-patient50-sims-gr-5-iterations-all-params.txt', sims_gr)
np.savetxt('somvanshiModel_output/somvanshiModel-nelson-patient50-sims-crh-5-iterations-all-params.txt', sims_crh)
np.savetxt('somvanshiModel_output/somvanshiModel-nelson-patient50-sims-acth-5-iterations-all-params.txt', sims_acth)
np.savetxt('somvanshiModel_output/somvanshiModel-nelson-patient50-sims-cort-5-iterations-all-params.txt', sims_cort)

In [ ]:
np.savetxt('somvanshiModel_output/somvanshiModel-nelson-patient50-initial-conditions-5-iterations-all-params.txt', y0)
np.savetxt('somvanshiModel_output/somvanshiModel-nelson-patient50-bounds-5-iterations-all-params.txt', bounds)

## Compute Means and Std Devations of Parameters and Output as Table <a name="paramtable"></a>

In [ ]:
Ki_mean = np.mean(opt_pars[:,1])
Ki_std = np.std(opt_pars[:,1])
r_mean = np.mean(opt_pars[:,2])
r_std = np.std(opt_pars[:,2])
K_strs_mean = np.mean(opt_pars[:,3])
K_strs_std = np.std(opt_pars[:,3])
n2_mean = np.mean(opt_pars[:,4])
n2_std = np.std(opt_pars[:,4])
k_crh_mean = np.mean(opt_pars[:,5])
k_crh_std = np.std(opt_pars[:,5])
V_acth1_mean = np.mean(opt_pars[:,6])
V_acth1_std = np.std(opt_pars[:,6])
k_acth_mean = np.mean(opt_pars[:,7])
k_acth_std = np.std(opt_pars[:,7])
k_cort_mean = np.mean(opt_pars[:,8])
k_cort_std = np.std(opt_pars[:,8])
V_cort_mean = np.mean(opt_pars[:,9])
V_cort_std = np.std(opt_pars[:,9])
k_dex1_mean = np.mean(opt_pars[:,10])
k_dex1_std = np.std(opt_pars[:,10])
V_crh_mean = np.mean(opt_pars[:,11])
V_crh_std = np.std(opt_pars[:,11])
V_acth2_mean = np.mean(opt_pars[:,12])
V_acth2_std = np.std(opt_pars[:,12])
Km_tnf3_mean = np.mean(opt_pars[:,13])
Km_tnf3_std = np.std(opt_pars[:,13])
V_starp2_mean = np.mean(opt_pars[:,14])
V_starp2_std = np.std(opt_pars[:,14])
k_starp_mean = np.mean(opt_pars[:,15])
k_starp_std = np.std(opt_pars[:,15])
V_starp1_mean = np.mean(opt_pars[:,16])
V_starp1_std = np.std(opt_pars[:,16])
Kil6_mean = np.mean(opt_pars[:,17])
Kil6_std = np.std(opt_pars[:,17])
k_dex2_mean = np.mean(opt_pars[:,18])
k_dex2_std = np.std(opt_pars[:,18])
V_dex2_mean = np.mean(opt_pars[:,19])
V_dex2_std = np.std(opt_pars[:,19])
Ki_GR_mean = np.mean(opt_pars[:,20])
Ki_GR_std = np.std(opt_pars[:,20])
V_GRmrna_mean = np.mean(opt_pars[:,21])
V_GRmrna_std = np.std(opt_pars[:,21])
kon_mean = np.mean(opt_pars[:,22])
kon_std = np.std(opt_pars[:,22])
k_GRprot_mean = np.mean(opt_pars[:,23])
k_GRprot_std = np.std(opt_pars[:,23])
krt_mean = np.mean(opt_pars[:,24])
krt_std = np.std(opt_pars[:,24])
kre_mean = np.mean(opt_pars[:,25])
kre_std = np.std(opt_pars[:,25])
k_GRmrna_mean = np.mean(opt_pars[:,26])
k_GRmrna_std = np.std(opt_pars[:,26])
V_GRprot_mean = np.mean(opt_pars[:,27])
V_GRprot_std = np.std(opt_pars[:,27])
fr_mean = np.mean(opt_pars[:,28])
fr_std = np.std(opt_pars[:,28])
tc_mean = np.mean(opt_pars[:,29])
tc_std = np.std(opt_pars[:,29])
ki_tnf_mean = np.mean(opt_pars[:,30])
ki_tnf_std = np.std(opt_pars[:,30])
n3_mean = np.mean(opt_pars[:,31])
n3_std = np.std(opt_pars[:,31])
n4_mean = np.mean(opt_pars[:,32])
n4_std = np.std(opt_pars[:,32])
fac_mean = np.mean(opt_pars[:,33])
fac_std = np.std(opt_pars[:,33])
k_lps_mean = np.mean(opt_pars[:,34])
k_lps_std = np.std(opt_pars[:,34])
V_phg1_mean = np.mean(opt_pars[:,35])
V_phg1_std = np.std(opt_pars[:,35])
V_phg2_mean = np.mean(opt_pars[:,36])
V_phg2_std = np.std(opt_pars[:,36])
Km_tnf1_mean = np.mean(opt_pars[:,37])
Km_tnf1_std = np.std(opt_pars[:,37])
Ki_tgf1_mean = np.mean(opt_pars[:,38])
Ki_tgf1_std = np.std(opt_pars[:,38])
Ki_IL10_mean = np.mean(opt_pars[:,39])
Ki_IL10_std = np.std(opt_pars[:,39])
k_phg_mean = np.mean(opt_pars[:,40])
k_phg_std = np.std(opt_pars[:,40])
V_tgf1_mean = np.mean(opt_pars[:,41])
V_tgf1_std = np.std(opt_pars[:,41])
k_tgf_mean = np.mean(opt_pars[:,42])
k_tgf_std = np.std(opt_pars[:,42])
V_tnf1_mean = np.mean(opt_pars[:,43])
V_tnf1_std = np.std(opt_pars[:,43])
Km_phg1_mean = np.mean(opt_pars[:,44])
Km_phg1_std = np.std(opt_pars[:,44])
Ki_tgf2_mean = np.mean(opt_pars[:,45])
Ki_tgf2_std = np.std(opt_pars[:,45])
V_tnf2_mean = np.mean(opt_pars[:,46])
V_tnf2_std = np.std(opt_pars[:,46])
Km_tnf2_mean = np.mean(opt_pars[:,47])
Km_tnf2_std = np.std(opt_pars[:,47])
k_tnf_mean = np.mean(opt_pars[:,48])
k_tnf_std = np.std(opt_pars[:,48])
Km_il61_mean = np.mean(opt_pars[:,49])
Km_il61_std = np.std(opt_pars[:,49])
Km_phg2_mean = np.mean(opt_pars[:,50])
Km_phg2_std = np.std(opt_pars[:,50])
k_il10_mean = np.mean(opt_pars[:,51])
k_il10_std = np.std(opt_pars[:,51])
V_il102_mean = np.mean(opt_pars[:,52])
V_il102_std = np.std(opt_pars[:,52])
Km_tgf3_mean = np.mean(opt_pars[:,53])
Km_tgf3_std = np.std(opt_pars[:,53])
V_il101_mean = np.mean(opt_pars[:,54])
V_il101_std = np.std(opt_pars[:,54])
Ki_il102_mean = np.mean(opt_pars[:,55])
Ki_il102_std = np.std(opt_pars[:,55])
V_il62_mean = np.mean(opt_pars[:,56])
V_il62_std = np.std(opt_pars[:,56])
Km_il6tnf_mean = np.mean(opt_pars[:,57])
Km_il6tnf_std = np.std(opt_pars[:,57])
Km_phg3_mean = np.mean(opt_pars[:,58])
Km_phg3_std = np.std(opt_pars[:,58])
V_il61_mean = np.mean(opt_pars[:,59])
V_il61_std = np.std(opt_pars[:,59])
k_il6_mean = np.mean(opt_pars[:,60])
k_il6_std = np.std(opt_pars[:,60])
tp_mean = np.mean(opt_pars[:,61])
tp_std = np.std(opt_pars[:,61])
tnf_b_mean = np.mean(opt_pars[:,62])
tnf_b_std = np.std(opt_pars[:,62])
il6_b_mean = np.mean(opt_pars[:,63])
il6_b_std = np.std(opt_pars[:,63])
V_tgf2_mean = np.mean(opt_pars[:,64])
V_tgf2_std = np.std(opt_pars[:,64])
n6_mean = np.mean(opt_pars[:,65])
n6_std = np.std(opt_pars[:,65])
n7_mean = np.mean(opt_pars[:,66])
n7_std = np.std(opt_pars[:,66])
n8_mean = np.mean(opt_pars[:,67])
n8_std = np.std(opt_pars[:,67])
n9_mean = np.mean(opt_pars[:,68])
n9_std = np.std(opt_pars[:,68])
n10_mean = np.mean(opt_pars[:,69])
n10_std = np.std(opt_pars[:,69])

In [ ]:
print(tabulate([["Ki", "%f +- %f" % (Ki_mean, Ki_std)], ["r", "%f +- %f" % (r_mean, r_std)], ["K_strs", "%f +- %f" % (K_strs_mean, K_strs_std)], ["n2", "%f +- %f" % (n2_mean, n2_std)], ["k_crh", "%f +- %f" % (k_crh_mean, k_crh_std)], ["V_acth1", "%f +- %f" % (V_acth1_mean, V_acth1_std)], ["k_acth", "%f +- %f" % (k_acth_mean, k_acth_std)], ["k_cort", "%f +- %f" % (k_cort_mean, k_cort_std)], ["V_cort", "%f +- %f" % (V_cort_mean, V_cort_std)], ["k_dex1", "%f +- %f" % (k_dex1_mean, k_dex1_std)], ["V_crh", "%f +- %f" % (V_crh_mean, V_crh_std)], ["V_acth2", "%f +- %f" % (V_acth2_mean, V_acth2_std)], ["Km_tnf3", "%f +- %f" % (Km_tnf3_mean, Km_tnf3_std)], ["V_starp2", "%f +- %f" % (V_starp2_mean, V_starp2_std)], ["k_starp", "%f +- %f" % (k_starp_mean, k_starp_std)], ["V_starp1", "%f +- %f" % (V_starp1_mean, V_starp1_std)], ["Kil6", "%f +- %f" % (Kil6_mean, Kil6_std)], ["k_dex2", "%f +- %f" % (k_dex2_mean, k_dex2_std)], ["V_dex2", "%f +- %f" % (V_dex2_mean, V_dex2_std)], ["Ki_GR", "%f +- %f" % (Ki_GR_mean, Ki_GR_std)], ["V_GRmrna", "%f +- %f" % (V_GRmrna_mean, V_GRmrna_std)], ["kon", "%f +- %f" % (kon_mean, kon_std)], ["k_GRprot", "%f +- %f" % (k_GRprot_mean, k_GRprot_std)], ["krt", "%f +- %f" % (krt_mean, krt_std)], ["kre", "%f +- %f" % (kre_mean, kre_std)], ["k_GRmrna", "%f +- %f" % (k_GRmrna_mean, k_GRmrna_std)], ["V_GRprot", "%f +- %f" % (V_GRprot_mean, V_GRprot_std)], ["fr", "%f +- %f" % (fr_mean, fr_std)], ["tc", "%f +- %f" % (tc_mean, tc_std)], ["ki_tnf", "%f +- %f" % (ki_tnf_mean, ki_tnf_std)], ["n3", "%f +- %f" % (n3_mean, n3_std)], ["n4", "%f +- %f" % (n4_mean, n4_std)], ["fac", "%f +- %f" % (fac_mean, fac_std)], ["k_lps", "%f +- %f" % (k_lps_mean, k_lps_std)], ["V_phg1", "%f +- %f" % (V_phg1_mean, V_phg1_std)], ["V_phg2", "%f +- %f" % (V_phg2_mean, V_phg2_std)], ["Km_tnf1", "%f +- %f" % (Km_tnf1_mean, Km_tnf1_std)], ["Ki_tgf1", "%f +- %f" % (Ki_tgf1_mean, Ki_tgf1_std)], ["Ki_IL10", "%f +- %f" % (Ki_IL10_mean, Ki_IL10_std)], ["k_phg", "%f +- %f" % (k_phg_mean, k_phg_std)], ["V_tgf1", "%f +- %f" % (V_tgf1_mean, V_tgf1_std)], ["k_tgf", "%f +- %f" % (k_tgf_mean, k_tgf_std)], ["V_tnf1", "%f +- %f" % (V_tnf1_mean, V_tnf1_std)], ["Km_phg1", "%f +- %f" % (Km_phg1_mean, Km_phg1_std)], ["Ki_tgf2", "%f +- %f" % (Ki_tgf2_mean, Ki_tgf2_std)], ["V_tnf2", "%f +- %f" % (V_tnf2_mean, V_tnf2_std)], ["Km_tnf2", "%f +- %f" % (Km_tnf2_mean, Km_tnf2_std)], ["k_tnf", "%f +- %f" % (k_tnf_mean, k_tnf_std)], ["Km_il61", "%f +- %f" % (Km_il61_mean, Km_il61_std)], ["Km_phg2", "%f +- %f" % (Km_phg2_mean, Km_phg2_std)], ["k_il10", "%f +- %f" % (k_il10_mean, k_il10_std)], ["V_il102", "%f +- %f" % (V_il102_mean, V_il102_std)], ["Km_tgf3", "%f +- %f" % (Km_tgf3_mean, Km_tgf3_std)], ["V_il101", "%f +- %f" % (V_il101_mean, V_il101_std)], ["Ki_il102", "%f +- %f" % (Ki_il102_mean, Ki_il102_std)], ["V_il62", "%f +- %f" % (V_il62_mean, V_il62_std)], ["Km_il6tnf", "%f +- %f" % (Km_il6tnf_mean, Km_il6tnf_std)], ["Km_phg3", "%f +- %f" % (Km_phg3_mean, Km_phg3_std)], ["V_il61", "%f +- %f" % (V_il61_mean, V_il61_std)], ["k_il6", "%f +- %f" % (k_il6_mean, k_il6_std)], ["tp", "%f +- %f" % (tp_mean, tp_std)], ["tnf_b", "%f +- %f" % (tnf_b_mean, tnf_b_std)], ["il6_b", "%f +- %f" % (il6_b_mean, il6_b_std)], ["V_tgf2", "%f +- %f" % (V_tgf2_mean, V_tgf2_std)], ["n6", "%f +- %f" % (n6_mean, n6_std)], ["n7", "%f +- %f" % (n7_mean, n7_std)], ["n8", "%f +- %f" % (n8_mean, n8_std)], ["n9", "%f +- %f" % (n9_mean, n9_std)], ["n10", "%f +- %f" % (n10_mean, n10_std)]], headers = ["Parameter", "Mean +- Standard Deviation"]))

In [ ]:
np.savetxt('somvanshiModel_output/somvanshiModel-nelson-patient50-param-means-stds-5-iterations-all-params.txt', [Ki_mean, Ki_std, r_mean, r_std, Ki_mean, Ki_std, r_mean, r_std, K_strs_mean, K_strs_std, n2_mean, n2_std, k_crh_mean, k_crh_std, V_acth1_mean, V_acth1_std, k_acth_mean, k_acth_std, k_cort_mean, k_cort_std, V_cort_mean, V_cort_std, k_dex1_mean, k_dex1_std, V_crh_mean, V_crh_std, V_acth2_mean, V_acth2_std, Km_tnf3_mean, Km_tnf3_std, V_starp2_mean, V_starp2_std, k_starp_mean, k_starp_std, V_starp1_mean, V_starp1_std, Kil6_mean, Kil6_std, k_dex2_mean, k_dex2_std, V_dex2_mean, V_dex2_std, Ki_GR_mean, Ki_GR_std, V_GRmrna_mean, V_GRmrna_std, kon_mean, kon_std, k_GRprot_mean, k_GRprot_std, krt_mean, krt_std, kre_mean, kre_std, k_GRmrna_mean, k_GRmrna_std, V_GRprot_mean, V_GRprot_std, fr_mean, fr_std, tc_mean, tc_std, ki_tnf_mean, ki_tnf_std, n3_mean, n3_std, n4_mean, n4_std, fac_mean, fac_std, k_lps_mean, k_lps_std, V_phg1_mean, V_phg1_std, V_phg2_mean, V_phg2_std, Km_tnf1_mean, Km_tnf1_std, Ki_tgf1_mean, Ki_tgf1_std, Ki_IL10_mean, Ki_IL10_std, k_phg_mean, k_phg_std, V_tgf1_mean, V_tgf1_std, k_tgf_mean, k_tgf_std, V_tnf1_mean, V_tnf1_std, Km_phg1_mean, Km_phg1_std, Ki_tgf2_mean, Ki_tgf2_std, V_tnf2_mean, V_tnf2_std, Km_tnf2_mean, Km_tnf2_std, k_tnf_mean, k_tnf_std, Km_il61_mean, Km_il61_std, Km_phg2_mean, Km_phg2_std, k_il10_mean, k_il10_std, V_il102_mean, V_il102_std, Km_tgf3_mean, Km_tgf3_std, V_il101_mean, V_il101_std, Ki_il102_mean, Ki_il102_std, V_il62_mean, V_il62_std, Km_il6tnf_mean, Km_il6tnf_std, Km_phg3_mean, Km_phg3_std, V_il61_mean, V_il61_std, k_il6_mean, k_il6_std, tp_mean, tp_std, tnf_b_mean, tnf_b_std, il6_b_mean, il6_b_std, V_tgf2_mean, V_tgf2_std, n6_mean, n6_std, n7_mean, n7_std, n8_mean, n8_std, n9_mean, n9_std, n10_mean, n10_std])

[Back to Top](#top)

## Plots <a name="plots"></a>

In [ ]:
%matplotlib inline
# plot of raw data with simulation data for cortisol, then just simulation data for crh and acth

fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, figsize=(15,25))

#ax1.plot(optimizedSimData[:,0], sims_crh, label = "Mean Simulated CRH")
# for more than 1 iteration, uncomment below
ax1.plot(optimizedSimData[-237:,0] - 100, np.mean(sims_crh_100, axis = 1), label = "Mean Simulated CRH", color = "blue")
ax1.fill_between(optimizedSimData[-237:,0] - 100, np.mean(sims_crh_100, axis = 1) - np.std(sims_crh_100, axis = 1), np.mean(sims_crh_100, axis = 1) + np.std(sims_crh_100, axis = 1), alpha = 0.3, label = "Simulated CRH Standard Deviation")
ax1.legend(loc = "upper left", shadow = True, fancybox = True)
ax1.set(xlabel = "Time (h)", ylabel = "CRH (micrograms/dL)", title = "CRH Concentration")


# change this line when you change which data set you are matching
ax2.plot(nelson.ACTH[:,0], nelson.ACTH[:,1], label = "Nelson ACTH Data - Patient Mean", color = "orange")

# for more than 1 iteration, uncomment below and comment the following line
#ax2.plot(optimizedSimData[:,0], sims_acth, label = "Mean")
ax2.plot(optimizedSimData[:,0], np.mean(sims_acth, axis = 1), label = "Mean Simulated ACTH", color = "blue")
ax2.fill_between(optimizedSimData[:,0], np.mean(sim1s_acth, axis = 1) - np.std(sims_acth, axis = 1), np.mean(sims_acth, axis = 1) + np.std(sims_acth, axis = 1), alpha = 0.3, label = "Simulated ACTH Standard Deviation")
ax2.legend(loc = "upper right", shadow = True, fancybox = True)
ax2.set(xlabel = "Time (h)", ylabel = "ACTH (pg/mL)", title = "ACTH Concentration")


# change this line when you change which data set you are matching
ax3.plot(nelson.cortisol[:,0], nelson.cortisol[:,1], label = "Nelson Cortisol Data - Patient Mean", color = "orange")

# for more than 1 iteration, uncomment below and comment the following line
#ax3.plot(optimizedSimData[:,0], sims_cort, label = "Simulation Mean")
ax3.plot(optimizedSimData[:,0], np.mean(sims_cort, axis = 1), label = "Mean Simulated Cortisol", color = "blue")
ax3.fill_between(optimizedSimData[:,0], np.mean(sims_cort, axis = 1) - np.std(sims_cort, axis = 1), np.mean(sims_cort, axis = 1) + np.std(sims_cort, axis = 1), alpha = 0.3, label = "Simulated Cortisol Standard Deviation")
ax3.legend(loc = "upper left", shadow = True, fancybox = True)
ax3.set(xlabel = "Time (h)", ylabel = "Cortisol (micrograms/dL)", title = "Cortisol Concentration")

#ax4.plot(optimizedSimData[:,0], sims_gr, label = "Mean")
# for more than 1 iteration, uncomment below
ax4.plot(optimizedSimData[:,0], np.mean(sims_gr, axis = 1), label = "Mean Simulated Nuclear GR")
ax4.fill_between(optimizedSimData[:,0], np.mean(sims_gr, axis = 1) - np.std(sims_gr, axis = 1), np.mean(sims_gr, axis = 1) + np.std(sims_gr, axis = 1), alpha = 0.5, label = "Simulated GR Standard Deviation")
ax4.legend(loc = "upper left", shadow = True, fancybox = True)
ax4.set(xlabel = "Time (h)", ylabel = "GR (micrograms)", title = "Nuclear GR Concentration")

txt = "cost = [582.763, 587.116]"
plt.figtext(0.1, 0.09, txt, wrap = True, horizontalalignment = 'center', fontsize = 12)
plt.savefig("somvanshiModel_output/somvanshiModel-nelson-patientMean-5-iterations-all-params.png", dpi = 300)


[Back to Top](#top)

## No Optimization Run and Plot <a name="no-opt"></a>

In [ ]:
# the authors run the simulation for 1500 hours, because it takes several
# days to reach a stable oscillatory state
t_start = -0.01
t_end = 2.35
t_step = 0.01

# set the initial conditions
y0 = [17.4896364862443,nelson.ACTH[0,51],0.794555311165795, nelson.cortisol[0,51],0,0,11.1479571299593,20.1160322148203,342.786973832252,24.8798181706611,31.3302385380763,0.000129677801623061,5897.54196590606,25,0.446309929849742,1.00160367257638,22.8093965060946,1.09564141359877]

In [ ]:
%%time

# run the solver with authors' published parameters
optimizedSimData = model(authors_params, y0)
    
# save CRH, cortisol and ACTH data into sims arrays
sims_crh = optimizedSimData[:,1]
sims_acth = optimizedSimData[:,2]
sims_starp = optimizedSimData[:,3]
sims_cort = optimizedSimData[:,4]
sims_dex1 = optimizedSimData[:,5]
sims_dex2 = optimizedSimData[:,6]
sims_cortp = optimizedSimData[:,7]
sims_gr_mrna = optimizedSimData[:,8]
sims_gr_prot = optimizedSimData[:,9]
sims_gr_cyt = optimizedSimData[:,10]
sims_gr = optimizedSimData[:,11]
sims_lps = optimizedSimData[:,12]
sims_phg = optimizedSimData[:,13]
sims_phg1 = optimizedSimData[:,14]
sims_tgf = optimizedSimData[:,15]
sims_tnf = optimizedSimData[:,16]
sims_il10 = optimizedSimData[:,17]
sims_il6 = optimizedSimData[:,18]

In [ ]:
print(costFun.SSE_cost(nelson.ACTH[:,0], nelson.ACTH[:,51], nelson.cortisol[:,0], nelson.cortisol[:,51],optimizedSimData))

In [ ]:
sims_crh_100 = optimizedSimData[-2428:,1]
sims_acth_100 = optimizedSimData[-2428:,2]
sims_cort_100 = optimizedSimData[-2428:,4]
sims_gr_100 = optimizedSimData[-2428:,11]

## Plot the No-Optimization Simulation <a name="plotNoOpt" />

In [ ]:
%matplotlib inline
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows = 4, figsize = (15,25))

ax1.plot(optimizedSimData[-2428:,0]-100, sims_crh_100, label = "CRH", color = "orange")
ax1.set(ylabel = "CRH (micrograms/dL)", xlabel = "Time (hours)", title = "CRH Concentration")
ax1.legend(loc = "upper right", shadow = True, fancybox = True)
#ax1.set_xlim(675, 775)

#ax2.plot(golier.PTSDACTH[:,0], golier.PTSDACTH[:,1], label = "Golier PTSD Group ACTH")
ax2.plot(optimizedSimData[-2428:,0]-100, sims_acth_100, label = "ACTH")
ax2.set(ylabel = "ACTH (pg/mL)", xlabel = "Time (hours)", title = "ACTH Concentration")
ax2.legend(loc = "upper right", shadow = True, fancybox = True)
#ax2.set_xlim(675, 775)

#ax3.plot(golier.PTSDCortisol[:,0], golier.PTSDCortisol[:,1], label = "Golier PTSD Group Cortisol")
ax3.plot(optimizedSimData[-2428:,0]-100, sims_cort_100, label = "Cortisol")
ax3.set(ylabel = "Cortisol (micrograms/dL)", xlabel = "Time (hours)", title = "Cortisol Concentration")
ax3.legend(loc = "upper right", shadow = True, fancybox = True)
#ax3.set_xlim(675, 775)

ax4.plot(optimizedSimData[-2428:,0]-100, sims_gr_100, label = "Nuclear GR", color = "orange")
ax4.set(ylabel = "GR", xlabel = "Time (hours)", title = "Nuclear GR Concentration")
ax4.legend(loc = "upper right", shadow = True, fancybox = True)
#ax4.set_xlim(675, 775)

#txt = "cost = [2626.85, 2666.69]"
#plt.figtext(0.1, 0.09, txt, wrap = True, horizontalalignment = 'center', fontsize = 12)
#plt.savefig("somvanshiModel_output/somvanshiModel-golierPTSD-5-iterations-all-params-100-hour-prerun.png", dpi = 300)


[Back to Top](#top)

## Dependencies <a name="dependencies"></a>

In [ ]:
%load_ext watermark

In [ ]:
%watermark --iversions

[Back to Top](#top)